<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/OpenAltimetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAltimetry ICESat-2: Elevation Workflow
Overview


This notebook pulls data from ICESat-2 surface-specific products - ATL06, ATL07, ATL08, ATL10, ATL12, ATL13 via the OpenAltimetry API.


Instructions
Replace the "OA_API_URL" string with the data copied from : "Copy API URL" from the OpenAltimetry elevation chart and start visualizing your data!
Author: Minh Phan - mnphan@ucsd.edu. Based off the work of Luis Lopez - luis.lopez@nsidc.org

In [ ]:
import json
import requests

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447

OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json'

OA_API_URL += '&client=jupyter';

print(OA_API_URL)

r = requests.get(OA_API_URL)
data = r.json()


In [ ]:
data

[dates and track id google sheet](https://docs.google.com/spreadsheets/d/13sjRCD2E2oyjkJwXOxeu5P3V6Ql9nW0KZe1gnsPFNOM/edit?usp=sharing)

In [ ]:
a= [
["2018-10-23",386],
["2018-10-27",447],
["2018-11-06",592],
["2018-11-10",653],
["2018-11-14",714],
["2018-11-25",889],
["2018-11-29",950],
["2018-12-03",1011],
["2018-12-05",1034],
["2018-12-13",1156],
["2018-12-24",1331],
["2018-12-28",5],
["2019-01-01",66],
["2019-01-03",89],
["2019-01-07",150],
["2019-01-11",211],
["2019-01-22",386],
["2019-01-26",447],
["2019-02-05",592],
["2019-02-09",653],
["2019-02-13",714],
["2019-02-24",889],
["2019-02-28",950],
["2019-03-04",1011],
["2019-03-06",1034],
["2019-03-10",1095],
["2019-03-14",1156],
["2019-03-25",1331],
["2019-03-29",5],
["2019-04-02",66],
["2019-04-04",89],
["2019-04-08",150],
["2019-04-12",211],
["2019-04-23",386],
["2019-04-27",447],
["2019-05-07",592],
["2019-05-11",653],
["2019-05-15",714],
["2019-05-26",889],
["2019-05-30",950],
["2019-06-03",1011],
["2019-06-04",1034],
["2019-06-05",1034],
["2019-06-08",1095],
["2019-06-09",1095],
["2019-06-12",1156],
["2019-06-13",1156],
["2019-06-24",1331],
["2019-07-11",211],
["2019-07-23",386],
["2019-07-27",447],
["2019-08-05",592],
["2019-08-09",653],
["2019-08-13",714],
["2019-08-25",889],
["2019-08-29",950],
["2019-09-02",1011],
["2019-09-03",1034],
["2019-09-07",1095],
["2019-09-11",1156],
["2019-09-23",1331],
["2019-09-27",5],
["2019-10-01",66],
["2019-10-02",89],
["2019-10-06",150],
["2019-10-10",211],
["2019-10-22",386],
["2019-10-26",447],
["2019-11-04",592],
["2019-11-08",653],
["2019-11-12",714],
["2019-11-24",889],
["2019-11-28",950],
["2019-12-02",1011],
["2019-12-03",1034],
["2019-12-07",1095],
["2019-12-11",1156],
["2019-12-23",1331],
["2019-12-27",5],
["2019-12-31",66],
["2020-01-01",89],
["2020-01-05",150],
["2020-01-09",211],
["2020-01-20",386],
["2020-01-21",386],
["2020-01-24",447],
["2020-01-25",447],
["2020-02-03",592],
["2020-02-07",653],
["2020-02-11",714],
["2020-02-22",889],
["2020-02-26",950],
["2020-03-01",1011],
["2020-03-03",1034],
["2020-03-07",1095],
["2020-03-11",1156],
["2020-03-22",1331],
["2020-03-26",5],
["2020-03-30",66],
["2020-04-01",89],
["2020-04-05",150],
["2020-04-09",211],
["2020-04-20",386],
["2020-04-24",447],
["2020-05-04",592],
["2020-05-08",653],
["2020-05-12",714],
["2020-05-23",889],
["2020-05-27",950],
["2020-05-31",1011],
["2020-06-02",1034],
["2020-06-06",1095],
["2020-06-10",1156],
["2020-06-21",1331],
["2020-06-25",5],
["2020-06-29",66],
["2020-07-01",89],
["2020-07-05",150],
["2020-07-09",211],
["2020-07-20",386],
["2020-07-24",447],
["2020-08-03",592],
["2020-08-07",653],
["2020-08-11",714],
["2020-08-22",889],
["2020-08-26",950],
["2020-08-30",1011],
["2020-09-01",1034],
["2020-09-05",1095],
["2020-09-09",1156],
["2020-09-20",1331],
["2020-09-24",5],
["2020-09-28",66],
["2020-09-29",89],
["2020-09-30",89],
["2020-10-03",150],
["2020-10-04",150],
["2020-10-07",211],
["2020-10-08",211],
["2020-10-19",386],
["2020-10-23",447],
["2020-11-01",592],
["2020-11-05",653],
["2020-11-09",714],
["2020-11-21",889],
["2020-11-25",950],
["2020-11-29",1011],
["2020-11-30",1034],
["2020-12-04",1095],
["2020-12-08",1156],
["2020-12-20",1331],
["2020-12-24",5],
["2020-12-28",66],
["2020-12-29",89],
["2021-01-02",150],
["2021-01-06",211],
["2021-01-18",386],
["2021-01-22",447],
["2021-01-31",592],
["2021-02-04",653],
["2021-02-08",714],
["2021-02-20",889],
["2021-02-24",950],
["2021-02-28",1011],
["2021-03-01",1034],
["2021-03-05",1095],
["2021-03-09",1156],
["2021-03-21",1331],
["2021-03-25",5],
["2021-03-29",66],
["2021-03-30",89],
["2021-04-03",150],
["2021-04-07",211],
["2021-04-19",386],
["2021-04-23",447],
["2021-05-02",592],
["2021-05-06",653],
["2021-05-10",714],
["2021-05-22",889],
["2021-05-26",950],
["2021-05-30",1011],
["2021-05-31",1034],
["2021-06-04",1095],
["2021-06-08",1156],
["2021-06-19",1331],
["2021-06-20",1331],
["2021-06-23",5],
["2021-06-24",5],
["2021-06-27",66],
["2021-06-28",66],
["2021-06-29",89],
["2021-07-03",150],
["2021-07-07",211],
["2021-07-22",447],
["2021-08-01",592],
["2021-08-05",653],
["2021-08-09",714],
["2021-08-20",889],
["2021-08-24",950],
["2021-08-28",1011],
["2021-08-30",1034],
["2021-09-03",1095],
["2021-09-07",1156],
["2021-09-18",1331],
["2021-09-22",5],
["2021-09-26",66],
["2021-09-28",89],
["2021-10-02",150],
["2021-10-06",211],
["2021-10-17",386],
["2021-10-21",447],
["2021-10-31",592],
["2021-11-04",653],
["2021-11-08",714],
["2021-11-19",889],
["2021-11-23",950],
["2021-11-27",1011],
["2021-11-29",1034],
["2021-12-03",1095],
["2021-12-07",1156],
["2021-12-18",1331],
["2021-12-22",5],
["2021-12-26",66],
["2021-12-28",89],
["2022-01-01",150],
["2022-01-05",211],
["2022-01-16",386],
["2022-01-20",447],
["2022-01-30",592],
["2022-02-03",653],
["2022-02-07",714],
["2022-02-18",889],
["2022-02-22",950],
["2022-02-26",1011],
["2022-02-27",1034],
["2022-02-28",1034],
["2022-03-03",1095],
["2022-03-04",1095],
["2022-03-07",1156],
["2022-03-08",1156],
["2022-03-19",1331],
["2022-03-23",5],
["2022-03-27",66],
["2022-03-28",89],
["2022-04-01",150],
["2022-04-17",386],
["2022-04-21",447],
["2022-04-30",592],
["2022-05-04",653],
["2022-05-08",714],
["2022-05-20",889],
["2022-05-24",950],
["2022-05-28",1011],
["2022-06-02",1095],
["2022-06-18",1331],
["2022-06-22",5],
["2022-06-26",66],
["2022-06-27",89],
["2022-07-01",150],
["2022-07-05",211],
["2022-07-17",386],
["2022-07-30",592],
["2022-08-03",653],
["2022-08-07",714],
["2022-08-19",889],
["2022-08-23",950],
["2022-08-27",1011],
["2022-09-21",5],
["2022-09-25",66],
["2022-09-26",89],
["2022-09-30",150],
["2022-10-04",211],]

In [ ]:
len(a)

##**ALT06**

In [ ]:
from pathlib import Path  



filepath = Path('out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot
from pathlib import Path  



filepath = Path('out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []
complete_dataset = []
# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA >>> ',b_url)
    r = requests.get(b_url)
    data = r.json()
    return data


b =[]
for i in range(len(a)):
  OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+a[i][0]+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(a[i][1])+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'
  OA_API_URL += '&client=jupyter';
  b.append(getData(OA_API_URL))

for elevation_data in b:
  product = elevation_data["product"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['beam'] for word in OA_BEAMS):
          series = []
          
          data_name = 'lat_lon_elev'
          if product == 'ATL08':
              data_name = 'lat_lon_elev_canopy'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2]
              })
              
          print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['beam']
              df_beams.append(df)


 
              
  print('df_beams len: ' + str(len(df_beams)))

# <<<<SAVING TO .csv FILE>>>>
pd.concat(df_beams).to_csv(filepath)

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


In [ ]:
pd.concat(df_beams)


https://openaltimetry.org

In [ ]:
a = !curl -X GET "https://openaltimetry.org/data/api/icesat2/atl06?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&trackId=447&client=portal&outputFormat=json" -H "accept: */*"

In [ ]:
a

In [ ]:
!wget https://openaltimetry.org/data/api/icesat2/getTracks?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&outputFormat=json -O a.json


##**ALT03**

In [ ]:
from pathlib import Path  



filepath = Path('out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot
from pathlib import Path  



filepath = Path('out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []
complete_dataset = []
# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA >>> ',b_url)
    r = requests.get(b_url)
    data = r.json()
    return data


b =[]
for i in range(len(a)):
  OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+a[i][0]+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(a[i][1])+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json'
  OA_API_URL += '&client=jupyter';
  b.append(getData(OA_API_URL))

for elevation_data in b:
  product = elevation_data["product"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['beam'] for word in OA_BEAMS):
          series = []
          
          data_name = 'lat_lon_elev'
          if product == 'ATL08':
              data_name = 'lat_lon_elev_canopy'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2]
              })
              
          print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['beam']
              df_beams.append(df)


 
              
  print('df_beams len: ' + str(len(df_beams)))

# <<<<SAVING TO .csv FILE>>>>
pd.concat(df_beams).to_csv(filepath)

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


Requesting data from OA >>>  https://openaltimetry.org/data/api/icesat2/atl03?date=2018-10-23&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&trackId=386&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json&client=jupyter
